In [1]:
# Load libs
import os, sys
from tqdm import tqdm
import pandas as pd
# Sklearn
from sklearn.model_selection import train_test_split, KFold
# Import tensorflow classes
from tensorflow.keras import backend as K

%matplotlib tk

2023-03-01 12:36:03.804829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 12:36:04.118395: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-01 12:36:04.804672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-01 12:36:04.804752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# Import EEGNet-related functions
baseDir = os.path.expanduser('~/Documents/Codes/vissat/') # Dima's path
# sys.path.insert(0, os.path.expanduser('~/Documents/Codes/')) # Dima's path (uncomment and change if you need to add MAA data manager codes)
from lib.models.eegnet import EEGNet
from lib.models.deep_utils import Params
from eeg_data_management.lib import transform_data as data #eeg_data_management is a name of the folder in sys.path that harbors the MAA Data manager

In [6]:
# House keeping
dataDir = os.path.expanduser('~/data/vissat/') # Dima's path
baseLogs = os.path.join(baseDir, 'vissat_logs') # Dima's path
baseModels = os.path.join(baseDir, 'vissat_models') # Dima's path
baseResults = os.path.join(baseDir, 'vissat_results') # Dima's path
if not os.path.isdir(baseResults):
    os.makedirs(baseResults)
# Define subjects (48 and 56 - starred)
subjects = [13, 14, 15, 16, 18, 21, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 52, 54, 56, 60, 62, 64]

In [7]:
# General Parameters
SAMPLING_RATE = 250
LOWF = 5 # Low bound for filtering data
HIGHF = 40 # High bound for filtering data
TMIN = 0 # At which time point from trigger to start epoch
TMAX = 3 # At which time point from trigger to stop epoch
triggers = {'7.5Hz': 1075, '10Hz': 1010, '12Hz': 1012} # Set of events to retrieve from datasets 
# number of keys in triggers will be used as a number of classes to classify
WDUR = 1 # Durations of time window to classify
TDELTA = 0.05 # Sliding window step for data augmentation
RSEED = 33 # random seed for shuffling the data 
TEST_SPLIT = 0.2 # test split = testsplit
VAL_SPLIT = 0.15 # validation split = (1 - testsplit)*testsplit

In [8]:
# EEGNet parameters
N_EPOCHS = 200 # Number of epochs to train
BATCH_SIZE = 30 # Number of batches to split the training data into
OPTIMIZER = 'nadam'
DROPOUT_RATE = 0.1
F1 = 12 # Frequency filter
D = 8 # Default spatial filter - is changed to nchannels*1.5 when calling EEGNet
NORM_RATE = 0.25 # Extent to which one wants to contran kernels in last dense layer

# Cross-subject design

If you want to see demo for within-subject design please do not execute what is below this cell but go further until the next title and execute what is below there

In [9]:
# Load data, filter them and epoch them
dataEpoched = []
idTrigger = []
print('Loading data...')
for isubject, subject in enumerate(tqdm(subjects)):
    # mneDataF = data.load_data(dataDir, 'vissat', subjects, channelsToGet=['parietal', 'occipital'], isFilt=True)
    mneDataFSub = data.load_data(dataDir, 'vissat', subject)
    dataEpochedSub, idTriggerSub = data.extract_epochs(mneDataFSub, triggers)
    dataEpoched.append(dataEpochedSub)
    idTrigger.append(idTriggerSub)

Loading data...


100%|██████████| 25/25 [00:09<00:00,  2.65it/s]


In [10]:
# Initialize parameters
samples = int(WDUR*SAMPLING_RATE)
kernLength = int(samples/2)
nchannels = dataEpoched[0].shape[1]
parameters = Params(Chans=nchannels, wdur=WDUR, samplingRate=SAMPLING_RATE, dropoutRate=DROPOUT_RATE,
                    kernLength=kernLength, F1=F1, D=int(nchannels*1.5),
                    F2=F1*int(nchannels*1.5), norm_rate=NORM_RATE, 
                    dropoutType='Dropout')

In [11]:
# Augment the data and create labels
augmented = data.transform_data(dataEpoched, WDUR, TDELTA, idTrigger, isPooled=True,
                                isSaccade=False)
(nndata, maskSub, classEpochs) = augmented
labels = data.create_labels(triggers, maskSub, classEpochs)

Augmenting data: creating 1-s long windows sliding with timestep of 0.05 sec
Obtained 25024 time windows for class 1
Obtained 24832 time windows for class 2
Obtained 25376 time windows for class 3


In [12]:
# Final preparations
nndata = data.ensure_nonzero_signals(nndata)
nndata = data.scale_data(nndata)

Scaling each epoch between -1 and 1


100%|██████████| 75232/75232 [00:09<00:00, 8221.86it/s]


In [13]:
# Then we will do kfold (code adapted from 
# https://stackoverflow.com/questions/66695848/kfold-cross-validation-in-tensorflow)

k_fold = KFold(n_splits=int(1/TEST_SPLIT), shuffle=True, random_state=RSEED)
seqNetModel = []
fittedHist = []
predicted = []
accuracies = []

parameters.add_parameters(isAugment=True, isSaccade=False, lowFilter=LOWF, 
            highFilter=HIGHF, wdur=WDUR, tdelta=TDELTA, randomSeed=RSEED, 
            testSplit=TEST_SPLIT, valSplit=VAL_SPLIT)

for ifold, (train, test) in enumerate(k_fold.split(nndata, labels)):
    # Folders to save models and results
    mPathSave = os.path.join(baseModels, 'test', ('fold' + str(ifold+1))) # model
    lPathSave = os.path.join(baseLogs, 'test', ('fold' + str(ifold+1))) # logs
    rPathSave = os.path.join(baseResults, 'test', ('fold' + str(ifold+1))) # results                                    
    # clear the session 
    K.clear_session()

    maskSubTest = labels[test,len(triggers.keys())]
    labelsTest = labels[test,:len(triggers.keys())]
    # Get validation set
    xtrain, xval, ytrain, yval = train_test_split(nndata[train],
                                                labels[train], 
                                                test_size=VAL_SPLIT,
                                                random_state=RSEED, shuffle=True)
    ytrain = ytrain[:,:len(triggers.keys())]
    yval = yval[:,:len(triggers.keys())]

    # calling the model and compile it 
    
    seqNetModel.append(EEGNet(len(triggers.keys()), parameters, design='cross',
                        dirModel=mPathSave, dirLog=lPathSave, dirResult=rPathSave))
    seqNetModel[ifold].compile(optimizer=OPTIMIZER)

    # Train the model
    fittedHist.append(seqNetModel[ifold].train(xtrain, ytrain, N_EPOCHS, 
                                                batchSize=BATCH_SIZE, 
                                                valData=(xval, yval),
                                                earlyStopping=True))
    # Predict
    predicted.append(seqNetModel[ifold].test(nndata[test], batchSize=BATCH_SIZE))
    acc_temp, accSub = seqNetModel[ifold].evaluate(predicted[ifold], labelsTest, 
                        maskSubTest=maskSubTest)
    accuracies.append(acc_temp)
    print(f'Accuracy of fold #{ifold+1} is {accuracies[ifold]:.02%}')

    seqNetModel[ifold].save_parameters(parameters)

f = pd.DataFrame(accuracies)
f.to_csv(os.path.join(os.path.dirname(rPathSave), 'accuracies.csv')) # save one dir up

2023-02-28 16:38:23.908527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 16:38:23.935702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 16:38:23.936188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-28 16:38:23.936668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

The training will stop when validation loss will start to decrease
Epoch 1/200


2023-02-28 16:38:26.229263: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


1706/1706 [==============================] - ETA: 0s - loss: 0.5994 - accuracy: 0.7364
Epoch 1: saving model to /home/dbryzgal/Documents/Codes/vissat/vissat_models/test/fold1/cp.ckpt
1706/1706 [==============================] - 17s 8ms/step - loss: 0.5994 - accuracy: 0.7364 - val_loss: 0.4678 - val_accuracy: 0.8066
Epoch 2/200
 248/1706 [===>..........................] - ETA: 11s - loss: 0.4900 - accuracy: 0.7915

KeyboardInterrupt: 

# Within-subject design

Here we will preserve the order of trials to come to simulate real-life situation during calibration - online test type of structure

In [6]:
# Load data, filter them and epoch them
dataEpoched = []
idTrigger = []
print('Loading data...')
for isubject, subject in enumerate(tqdm(subjects)):
    # mneDataF = data.load_data(dataDir, 'vissat', subjects, channelsToGet=['parietal', 'occipital'], isFilt=True)
    mneDataFSub = data.load_data(dataDir, 'vissat', subject)
    dataEpochedSub, idTriggerSub = data.extract_epochs(mneDataFSub, triggers)
    dataEpoched.append(dataEpochedSub)
    idTrigger.append(idTriggerSub)

Loading data...


100%|██████████| 25/25 [00:08<00:00,  2.87it/s]


In [7]:
# Initialize parameters
samples = int(WDUR*SAMPLING_RATE)
kernLength = int(samples/2)
nchannels = dataEpoched[0].shape[1]
params = Params(Chans=nchannels, Samples=samples, dropoutRate=DROPOUT_RATE, 
                    kernLength=kernLength, F1=F1, D=int(nchannels*1.5), 
                    F2=F1*int(nchannels*1.5), norm_rate=NORM_RATE, 
                    dropoutType='Dropout')

In [8]:
# Augment the data and create labels
augmented = data.transform_data(dataEpoched, WDUR, TDELTA, idTrigger, isPooled=False,
                                isSaccade=False)
(nndata, maskSub, classN) = augmented

labels = []
for isubject in range(len(subjects)):
    labels.append(data.create_labels(triggers, maskSub[isubject], classN[isubject]))

Augmenting data: creating 1-s long windows sliding with timestep of 0.05 sec
Obtained average 1001 time windows for class 1
Obtained average 993 time windows for class 2
Obtained average 1015 time windows for class 3


In [9]:
for isub in range(len(subjects)):
    nndata[isub] = data.ensure_nonzero_signals(nndata[isub])
    nndata[isub] = data.scale_data(nndata[isub])

Scaling each epoch between -1 and 1


100%|██████████| 2720/2720 [00:00<00:00, 7351.70it/s]


Scaling each epoch between -1 and 1


100%|██████████| 2720/2720 [00:00<00:00, 7518.06it/s]


Scaling each epoch between -1 and 1


...

100%|██████████| 3072/3072 [00:00<00:00, 7712.91it/s]


Scaling each epoch between -1 and 1


100%|██████████| 3072/3072 [00:00<00:00, 7755.26it/s]


Scaling each epoch between -1 and 1


100%|██████████| 3072/3072 [00:00<00:00, 7800.30it/s]


In [10]:
# Split dataset
print(f"Splitting each subject's dataset in train {(1-TEST_SPLIT):.0%} and test {TEST_SPLIT:.0%} datasets")

nndataTrain = []
nndataTest = []
labelsTrain = []
labelsTest = []
for isub in range(len(subjects)):
        xtrain, xtest, ytrain, ytest = train_test_split(nndata[isub], labels[isub], 
                                        test_size=TEST_SPLIT, random_state=RSEED, 
                                        shuffle=False)
        nndataTrain.append(xtrain)
        nndataTest.append(xtest)
        labelsTrain.append(ytrain[:,:3])
        labelsTest.append(ytest[:,:3])

Splitting each subject's dataset in train 80% and test 20% datasets


In [11]:
params.add_parameters(isAugment=True, isSaccade=False, lowFilter=LOWF, 
            highFilter=HIGHF, wdur=WDUR, tdelta=TDELTA, randomSeed=RSEED,
            testSplit=TEST_SPLIT, valSplit=VAL_SPLIT)

cnt = 0
accuracies = []

# Train networks
for isub, sub in enumerate(subjects):
    mPathSave = os.path.join(baseModels, 'test_ws', ('subj' + str(sub))) # model
    rPathSave = os.path.join(baseResults, 'test_ws', ('subj' + str(sub)))  # results
    lPathSave = os.path.join(baseLogs, 'test_ws', ('subj' + str(sub)))  # logs
    if not os.path.isdir(mPathSave):
        os.makedirs(mPathSave)
    if not os.path.isdir(rPathSave):
        os.makedirs(rPathSave)
    # clear the session 
    K.clear_session()
    print(f'Training network for subject {sub}')

    model = EEGNet(len(triggers.keys()), params, design='within',
                    dirModel=mPathSave, dirLog=lPathSave, dirResult=rPathSave)
    model.compile(optimizer = OPTIMIZER)

    modelHist = model.train(nndataTrain[isub], labelsTrain[isub], 
                        N_EPOCHS, batchSize=BATCH_SIZE,
                        valSplit=VAL_SPLIT, earlyStopping=True)

    predicted = model.test(nndataTest[isub], batchSize=BATCH_SIZE)
    acc_temp, _ = model.evaluate(predicted, labelsTest[isub])
    accuracies.append(acc_temp)
    print(f'Accuracy of model for subject#{sub} is {accuracies[cnt]:.02%}')

    model.save_parameters(params)

    cnt += 1

f = pd.DataFrame(accuracies)
f.to_csv(os.path.join(os.path.dirname(rPathSave), 'acc.csv'))


Training network for subject 13
The training will stop when validation loss will start to decrease
Epoch 1/200


2023-02-21 12:22:46.945547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 12:22:46.992368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 12:22:46.992575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 12:22:46.992991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

62/62 [==============================] - ETA: 0s - loss: 1.0032 - accuracy: 0.5008
Epoch 1: saving model to /home/dbryzgal/Documents/Codes/vissat/vissat_models/test_ws/subj13/cp.ckpt
62/62 [==============================] - 3s 16ms/step - loss: 1.0032 - accuracy: 0.5008 - val_loss: 1.0869 - val_accuracy: 0.4893
Epoch 2/200
59/62 [===========================>..] - ETA: 0s - loss: 0.8079 - accuracy: 0.6441
Epoch 2: saving model to /home/dbryzgal/Documents/Codes/vissat/vissat_models/test_ws/subj13/cp.ckpt
62/62 [==============================] - 1s 9ms/step - loss: 0.7994 - accuracy: 0.6495 - val_loss: 1.0795 - val_accuracy: 0.5780
Epoch 3/200
61/62 [============================>.] - ETA: 0s - loss: 0.6002 - accuracy: 0.7541
Epoch 3: saving model to /home/dbryzgal/Documents/Codes/vissat/vissat_models/test_ws/subj13/cp.ckpt
62/62 [==============================] - 1s 10ms/step - loss: 0.5991 - accuracy: 0.7550 - val_loss: 1.1218 - val_accuracy: 0.1590
Epoch 4/200
58/62 [===================

KeyboardInterrupt: 